## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos (filtrado a nivel filas)

In [ ]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ["completitud_fila", "situacion_vivienda_ok"]
col_eliminar_tarjetas = ["nivel_tarjeta"]

# Se ejecuta la operación
#df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
#df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [3]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


Limpieza de los datos (filtrado a nivel filas)

In [8]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,regla_pct_ingresos,regla_pct_ingresos,regla_pct_ingresos,regla_pct_ingresos,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,ok,ok,ok,ok,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,0.0,True,ok,ok,ok,ok,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,0.0,True,ok,ok,ok,ok,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,0.0,True,ok,ok,ok,ok,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,0.0,True,ok,ok,ok,ok,ok


In [ ]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtrado: {df_creditos.shape[0]}")
temp = df_creditos[df_creditos['edad'] < 90]


temp_c = temp[temp['antiguedad_empleado'] <= 50]  ## REVISAR LA CANTIDAD DE VALORES FILTRADOS
print(f"Filas después del filtrado: {temp_c.shape[0]}")

# Otro filtro posible:
# df_creditos['antiguedad_empleado'] > 50

## APLICAR LOS OTROS FILTROS PLANTEADOS, Y TAMBIÉN EN EL DATASET DE DATOS DE TARJETAS

# También por las reglas del negocio
# df[df.regla_pct_ingresos == 'err']



Integración de datos

In [ ]:
df_integrado = pd.merge(temp_c2, df_tarjetas, on='id_cliente', how='inner') #Unir los dataset sobre los que hayamos aplicado los filtros
coincidencias = df_integrado.shape[0]

print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Transformación de atributos

In [ ]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}
estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_civil
cambios_estado_credito = {
    0 : 'C', #Cancelado
    1 : 'P' # Pendiente cancelación
}
estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

## SE PUEDEN HACER OTRAS TRANSFORMACIONES, COMO CAMBIAR LOS INGRESOS DE LOS EMPLEADOS A RANGOS

df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_final.head()



# Sobre este resultado será necesario eliminar las columnas auxiliares



Exportación de metadatos